In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os

import torch
from ipywidgets import interact

# from qsr_learning.data.data import draw, generate_objects

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Test the `Entity` Class

In [ ]:
import math

from PIL import Image
from qsr_learning.entity import Entity

In [ ]:
canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
entity1 = Entity(
    name="octopus",
    frame_of_reference="absolute",
    p=(30, 30),
    theta=0 / 360 * 2 * math.pi,
    size=(32, 32),
)
entity1.draw(canvas, add_bbox=True, add_front=True)
entity2 = Entity(
    name="trophy",
    frame_of_reference="absolute",
    p=(60, 60),
    theta=90 / 360 * 2 * math.pi,
    size=(32, 32),
)
entity2.draw(canvas, add_bbox=True, add_front=True)

# Test the Relations

In [ ]:
from ipywidgets import interact
from qsr_learning.relation import above, below, left_of, right_of


@interact(
    frame_of_reference=(0, 1),
    x1=(0, 150),
    y1=(0, 150),
    theta1=(0, 360),
    x2=(0, 150),
    y2=(0, 150),
    theta2=(0, 360),
)
def test_spatial_relations(
    frame_of_reference=0, x1=64, y1=64, theta1=0, x2=128, y2=128, theta2=150
):
    canvas = Image.new("RGBA", (224, 224), (127, 127, 127, 127))
    entity1 = Entity(
        name="octopus",
        frame_of_reference={0: "absolute", 1: "intrinsic"}[frame_of_reference],
        p=(x1, y1),
        theta=theta1 / 360 * 2 * math.pi,
        size=(32, 32),
    )
    entity1.draw(canvas, add_bbox=True, add_front=True)
    entity2 = Entity(
        name="trophy",
        frame_of_reference={0: "absolute", 1: "intrinsic"}[frame_of_reference],
        p=(x2, y2),
        theta=theta2 / 360 * 2 * math.pi,
        size=(32, 32),
    )
    entity2.draw(canvas, add_bbox=True, add_front=True)
    display(canvas)

    for relation in [left_of, right_of, above, below]:
        if relation(entity1, entity2):
            print(entity1.name, relation.__name__, entity2.name)
    for relation in [left_of, right_of, above, below]:
        if relation(entity2, entity1):
            print(entity2.name, relation.__name__, entity1.name)

# Test the Samples

In [ ]:
from qsr_learning.data import draw_entities, generate_entities
from qsr_learning.entity import emoji_names

entities = generate_entities(
    entity_names=emoji_names,  # ["octopus", "trophy"],
    num_entities=2,
    frame_of_reference="absolute",
    w_range=(16, 64),
    h_range=(16, 64),
)
image = draw_entities(entities, add_bbox=True, add_front=False)
display(image)

# Test the Dataset

In [ ]:
import math

import torch
from ipywidgets import interact
from PIL import Image
from torch.utils.data import random_split

from qsr_learning.data import DRLDataset
from qsr_learning.entity import emoji_names

dataset = DRLDataset(
    entity_names=["octopus", "trophy", "frog", "ghost"],
    excluded_combinations=[("octopus", "trophy")],
    relation_names=["left_of", "right_of"],
    num_entities=2,
    frame_of_reference="intrinsic",
    w_range=(32, 32),
    h_range=(32, 32),
    theta_range=(0, 2 * math.pi),
    add_bbox=True,
    add_front=True,
    transform=None,
    canvas_size=(224, 224),
    num_samples=1000,
)
train_dataset, validation_dataset, test_dataset = random_split(
    dataset,
    [800, 100, 100],
    generator=torch.Generator().manual_seed(0),
)
subset = train_dataset

In [ ]:
@interact(idx=(0, len(subset) - 1))
def display_sample(idx=0):

    image_t, question_t, answer_t = subset[idx]
    image = Image.fromarray(
        (255 * (dataset.std.view(-1, 1, 1) * image_t + dataset.mean.view(-1, 1, 1)))
        .permute(1, 2, 0)
        .numpy()
        .astype("uint8")
    )
    head, relation, tail = question_t.tolist()
    question = (
        dataset.idx2word[head],
        dataset.idx2word[relation],
        dataset.idx2word[tail],
    )
    answer = bool(answer_t)
    display(image)
    print(question)
    print("Ground truth: ", answer)